In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector as db
import credentials
import pickle

In [2]:
%%time
#Establish connection to the MySQLDB
MySQLDB = db.connect(host = credentials.host, user = credentials.user, password = credentials.password,
                     db = credentials.db)

Wall time: 211 ms


In [9]:
#Two pickle files are used for this activity
#The test_indiv pickle contains intermediate records for each month's query
#The test_merged pickle contains all records across every month
pkl_indiv = open("test_indiv.pickle","wb")
pkl_merged = open("test_merged.pickle","wb")

def execute_query(lower,upper):
    """
    Execute a query for a given date range
    """
    sql_query = """
    select * from 
    (select * from 
    (select * from 
    (select year_week, business_sid, upper(trim(industry)) as industry, upper(trim(sub_industry)) as sub_industry,shipper,
    trim(service_type_description) as service_type,package_count, weight,shipment_date,delivery_date, delivery_time, 
    freight_charges,freight_discount_amount,misc_charges,misc_discount_amount, 
    net_charge_amount, zone, upper(trim(sender_city)) as sender_city, left(sender_zip,5) as sender_zip,
    upper(trim(recipient_city)) as recipient_city,left(recipient_zip,5) as recipient_zip 
    from libras.shipment_details 
    where sender_country = 'US' and recipient_country = 'US' and year_week >= {} and year_week < {}
    and delivery_date is not null 
    ) t1 
    where t1.shipment_date is not null) t2 
    where t2.freight_charges > 0) t3 
    where t3.zone is not null or trim(zone)!='' 
    limit 300000""".format(lower,upper)
    #print(sql_query)
    cursor = MySQLDB.cursor()
    cursor.execute(sql_query)
    records = pd.DataFrame(cursor.fetchall())
    records.to_pickle("test_indiv.pickle")
    return records 
    #print(len(records.index))

def store_results(monthly_records_df):
    """
    Store results from each query into a pickle
    """
    try:
        temp_df = pd.DataFrame()
        newDF = pd.read_pickle("test_merged.pickle")
        print("Merged pickle contains", len(newDF.index), "records before merge")
        temp_df = newDF.append(monthly_records_df)
        temp_df.to_pickle("test_merged.pickle")
        print("Merged pickle contains", len(temp_df.index), "records after merge")
    except EOFError:
        monthly_records_df.to_pickle("test_merged.pickle")
        print("Merged pickle contains", len(monthly_records_df.index), "records with initial merge")

In [ ]:
#Get records for 2018
min_date = 201823
max_date = 201852 #201852
lower_date_range = min_date
upper_date_range = lower_date_range + 4
while upper_date_range <=max_date:
    if upper_date_range <=201852:
        upper_date_range = lower_date_range + 4
        print("Getting records between year_week", lower_date_range,"and",upper_date_range, "(exclusive)") 
        store_results(execute_query(lower = lower_date_range,upper = upper_date_range))
        lower_date_range=upper_date_range

#Get records for 2019        
min_date = 201901
max_date = 201925 #201925
lower_date_range = min_date
upper_date_range = lower_date_range + 4
while upper_date_range <=max_date:
    if upper_date_range <=201925:
        upper_date_range = lower_date_range + 4
        print("Getting records between year_week", lower_date_range,"and",upper_date_range, "(exclusive)") 
        store_results(execute_query(lower = lower_date_range,upper = upper_date_range))
        lower_date_range=upper_date_range

Getting records between year_week  201823  and  201827  (exclusive)
Merged pickle contains  300000  records with initial merge
Getting records between year_week  201827  and  201831  (exclusive)
Merged pickle contains  300000  records before merge
Merged pickle contains  600000  records after merge
Getting records between year_week  201831  and  201835  (exclusive)
Merged pickle contains  600000  records before merge
Merged pickle contains  900000  records after merge
Getting records between year_week  201835  and  201839  (exclusive)
Merged pickle contains  900000  records before merge
Merged pickle contains  1200000  records after merge
Getting records between year_week  201839  and  201843  (exclusive)
Merged pickle contains  1200000  records before merge
Merged pickle contains  1500000  records after merge
Getting records between year_week  201843  and  201847  (exclusive)
Merged pickle contains  1500000  records before merge
Merged pickle contains  1800000  records after merge
Get

In [ ]:
unpickled_df = pd.read_pickle("test_merged.pickle")
len(unpickled_df)
unpickled_df.head()

In [2]:
# Code to replace zones 
unpickled_df = unpickled_df[unpickled_df.zone.apply(lambda x: x.isnumeric())]
unpickled_df['zone'] = unpickled_df['zone'].unpickled_df(int)
unpickled_df['zone'] = unpickled_df['zone'] % 100

NameError: name 'unpickled_df' is not defined